# Read dataset and plot graph

In [1]:
#pip install numpy
#pip install pandas
#pip install pandas-profiling
#pip install networkx
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import json
import networkx as nx
import matplotlib.pyplot as plt
from random import randrange

def read_dataset(path,drop_columns=None,keep_columns=None):
    #get rid of useless columns
    csv_data = pd.read_csv(path)
    
    if keep_columns != None:
        #keep only these columns
        return csv_data.filter(items=keep_columns)
    
    if drop_columns!= None:
        #drop these and keep the rest
        return csv_data.drop(drop_columns, axis=1)
    
    #finally, didn't drop or filter any column
    return csv_data     

def plot_graph(g,ds_nodes=[],attribute_nodes=[],feat_nodes=[],lit_nodes=[]):
    pos=nx.spring_layout(g)    
    nx.draw_networkx_nodes(g,pos,nodelist=ds_nodes,node_color="blue",node_size=900)
    nx.draw_networkx_nodes(g,pos,nodelist=attribute_nodes,node_color="green",node_size=900)
    nx.draw_networkx_nodes(g,pos,nodelist=feat_nodes,node_color="grey",node_size=900)
    nx.draw_networkx_nodes(g,pos,nodelist=lit_nodes,node_color="red",node_size=900)

    nx.draw_networkx_edges(g,pos,width=3)
    nx.draw_networkx_labels(g,pos,font_size=8)
    plt.show() 

## Graph  construction

In [2]:
def code_ds_id(data):
    return "DS_"+data
def code_attr_id(data,parent):
    return data+"|"+parent
def code_feat_id(data,parent):
    return data+"|"+parent
def code_literal_id(data,parent):
    return "literal_"+data+"|"+parent

In [3]:
def graph_dataset(datasets,g=None,instances=0):
    if g == None:
        g = nx.Graph()
    
    #create nodes and edges at datasetLevel
    features = datasets.columns[1:]
    
    if instances==0:
        number_instances = len(datasets)
    else:
        number_instances = instances
    
    for r in range(number_instances): 
        #node id is the openML id which is in the first column
        dataset_id = code_ds_id(str(datasets.iloc[r][0]))
        g.add_node(dataset_id,vector=bert("dataset"),tipo="dataset")
        row = datasets.iloc[r][1:]
        
        if instances == 0:
            number_features = len(features)
        else:
            number_features = min(instances,len(features))
            
        for i in range (number_features):
            feature_dataset_id = code_feat_id(features[i],dataset_id)
            literal_dataset_id = code_literal_id(str(i),dataset_id)
            g.add_node(feature_dataset_id,vector=bert("feature dataset"),tipo="feature dataset")
            g.add_node(literal_dataset_id,vector=bert(row[i]),tipo="literal dataset")
            g.add_edge(dataset_id,feature_dataset_id)
            g.add_edge(feature_dataset_id,literal_dataset_id)
            
    return g

In [4]:
def graph_attribute(datasets,g=None,instances=0):
    if g == None:
        g = nx.Graph()
        
    #create nodes and edges at datasetLevel
    features = datasets.columns[2:]
    
    if instances==0:
        number_instances = len(datasets)
    else:
        number_instances = min (instances,len(datasets))
    
    for r in range(number_instances): 
        #node id is the openML id which is in the first column
        #attr name is the 2nd column
        dataset_id = code_ds_id(str(datasets.iloc[r][0]))
        attribute_id = code_attr_id(datasets.iloc[r][1],dataset_id)
        row = datasets.iloc[r][2:]
        
        g.add_node(attribute_id,vector=bert("attribute"),tipo="attribute")
        
        #relation of dataset and an attribute
        g.add_edge(dataset_id,attribute_id)
        
        if instances == 0:
            number_features = len(features)
        else:
            number_features = min (instances,len(features))
            
        for i in range (number_features):
            feature_attribute_id = code_feat_id(features[i],attribute_id)
            literal_dataset_id = code_literal_id(str(i),attribute_id)
            g.add_node(feature_attribute_id,vector=bert("feature attribute"),tipo="feature attribute")
            g.add_node(literal_dataset_id,vector=bert(row[i]),tipo="literal attribute")
            g.add_edge(attribute_id,feature_attribute_id)
            g.add_edge(feature_attribute_id,literal_dataset_id)
            
    return g

# Test

In [12]:
df_dataset = read_dataset("./openml_203ds_datasets_index.csv",drop_columns=["Num", "dataset_topic"]);
g = g = nx.Graph()
g = graph_dataset(df_dataset,g,0)
df_attributes = read_dataset("./openml_203ds_attributes_nominal_index.csv",drop_columns=["dataset_name", "type_converted"]);
g = graph_attribute(df_attributes,g,0)
df_attributes_numeric = read_dataset("./openml_203ds_attributes_numeric_index.csv",drop_columns=["dataset_name", "type_converted"]);
g = graph_attribute(df_attributes_numeric,g,0)
#plot_graph(g,ds_nodes,attr_nodes,feat_nodes,lit_nodes);



## Write and read graph in file

In [21]:
nx.write_gpickle(g, "encoded_features.gpickle")

In [8]:
g = nx.read_gpickle("encoded_features.gpickle")

## Get array of ids

In [ ]:
datasets = [x for x,y in g.nodes(data=True) if y['tipo']=="dataset"]
datasets_features = [x for x,y in g.nodes(data=True) if y['tipo']=="feature dataset"]
datasets_literals = [x for x,y in g.nodes(data=True) if y['tipo']=="literal dataset"]
attributes = [x for x,y in g.nodes(data=True) if y['tipo']=="attribute"]
attributes_features = [x for x,y in g.nodes(data=True) if y['tipo']=="feature attribute"]
attributes_literals = [x for x,y in g.nodes(data=True) if y['tipo']=="literal attribute"]

In [ ]:
print(len(datasets))
print(len(datasets_features))
print(len(datasets_literals))
print(len(attributes))
print(len(attributes_features))
print(len(attributes_literals))

## Get nodes and print data from nodes

In [13]:
sample = [y for x,y in g.nodes(data=True) if y['tipo']=="dataset"]
print(sample[0]["vector"])

tensor([-5.6158e-01,  4.4368e-01, -1.7006e-01, -1.9077e-01,  1.5807e-01,
        -2.6305e-02, -6.4893e-02,  2.8374e-01,  3.2929e-01,  5.7418e-01,
         2.8632e-01,  1.7504e-02,  3.3841e-01, -1.9123e-01, -3.9816e-01,
        -1.0002e+00,  9.1957e-02,  5.1864e-01,  3.5536e-01,  2.9942e-01,
         6.1362e-01, -2.6814e-01,  3.7264e-01, -2.9228e-02,  4.1818e-01,
        -2.7104e-01,  1.6840e-01, -3.3542e-01, -2.1305e-01,  4.5572e-01,
         5.5157e-02, -2.0026e-01, -4.1137e-02,  3.9757e-02,  9.4895e-02,
         8.2463e-02, -3.9485e-01,  4.0088e-01,  3.0982e-01, -2.6520e-01,
        -3.5821e-01, -3.1978e-01, -2.1658e-01,  1.2864e-01,  5.1229e-01,
        -1.7002e-01,  2.2455e-01,  3.5818e-02, -6.6891e-01,  2.9675e-02,
        -6.4308e-01,  1.8412e-01, -2.8592e-01,  1.7483e-01, -5.0626e-01,
        -4.2068e-01, -1.4417e-02,  5.5719e-02, -2.9205e-01,  5.3836e-01,
         1.0070e-01, -2.3647e-01, -3.4717e-02,  4.2593e-01,  4.9275e-01,
         9.7349e-02, -2.4548e-01, -1.7963e-01,  4.1

## Possitive and negative samples

In [ ]:
def get_samples(df):
    possitive_pairs = []
    negative_pairs = []
    for index, row in df.iterrows():
        if row[2] == 1:
            possitive_pairs.append((row[0],row[1]))
        else:
            if randrange(1, 10) > 8:
                negative_pairs.append((row[0],row[1]))
    return possitive_pairs,negative_pairs

In [ ]:
df_matching = read_dataset("./openml_203ds_datasets_matching.csv",keep_columns=["'dataset1_id'", "'dataset2_id'","'matching_topic'"]);
pos,neg = get_samples(df_matching)
print("Possitive samples: "+str(len(pos)) + " Negative samples (10%): "+str(len(neg)))


## Convert strings to single words

In [6]:
def is_number(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [7]:
import re
def get_tokens(nodes):
    #nodes_unique = set(nodes)
    nodes_unique = nodes
    numerical=[]
    nominal=[]
    for s in nodes_unique:
        if is_number(s):
            numerical.append(float(s))
        else:
            #nominal = nominal +  list(map(lambda a : re.sub(r'[^A-Za-z0-9 \']+','',a).lower(),(re.split('[-_;|]\s*',s))))
            nominal.append(re.sub(r'[^A-Za-z0-9 \']+',' ',s).lower())
            #nominal = nominal +  s.split("|")
    return list(set(numerical)),list(set(nominal))

In [8]:
def vector_from_number(n):
    #get the embeddings for numerical values that make sense
    out = []
    return out

In [ ]:
vec_num,vec_nom = get_tokens(lit_nodes)

In [ ]:
for i in range(100):
    print("num: "+ str(vec_num[i]))
    print("nom: "+ vec_nom[i])

## cosine similarity

In [18]:
from scipy.spatial.distance import cosine
def cosine_vectors(vec1,vec2):
    return (1- cosine(vec1,vec2))

## Fasttext

In [ ]:
#pip install fasttext
import fasttext
#fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('../models/cc.en.300.bin')
print(ft.get_dimension())

In [ ]:
def fasttex(value):
    if is_number(value):
        value = str(value)

    return ft.get_sentence_vector(value)

In [ ]:
def fasttex_vectors(nodes_literals):
    nodes_vector = []
    for literal in nodes_literals:
        
        if is_number(literal):
            literal = str(literal)
        
        nodes_vector.append(ft.get_sentence_vector(literal))
        
    return nodes_vector

## Bert


In [10]:
#pip install torch==1.4.0+cpu torchvision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
#pip install transformers
from transformers import BertModel, BertTokenizer
import torch

#load model in memory
tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-12_H-768_A-12')
model = BertModel.from_pretrained('google/bert_uncased_L-12_H-768_A-12')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased')
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [11]:
def bert(value):
    if is_number(value):
        value = str(value)

    #add special tokens at the begining and end, and takes until 512 tokens max 
    tokenized = tokenizer.encode(value, add_special_tokens=True,max_length=512)
    input_ids = torch.tensor(tokenized).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)

    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    #result shape: (batch size, sequence length, model hidden dimension)
    #print(last_hidden_states.shape)

    #make the mean of the vectors to have 1 vector for the whole sentence and store result
    return torch.mean(last_hidden_states[0],dim=0).detach()

In [ ]:
def bert_vectors(nodes_literals):
    nodes_vector = []
    for literal in nodes_literals:
        
        if is_number(literal):
            literal = str(literal)
        
        #add special tokens at the begining and end, and takes until 512 tokens max 
        tokenized = tokenizer.encode(literal, add_special_tokens=True,max_length=512)
        input_ids = torch.tensor(tokenized).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids)

        last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
        #result shape: (batch size, sequence length, model hidden dimension)
        #print(last_hidden_states.shape)
        
        #make the mean of the vectors to have 1 vector for the whole sentence and store result
        nodes_vector.append(torch.mean(last_hidden_states[0],dim=0).detach()) 
    return nodes_vector

## Get embeddings of words and calculate similarity

In [ ]:
s1= "University degree"
s2="Bachelor"
vectors_b = bert_vectors([s1,s2])
vectors_ft = fasttex_vectors([s1,s2])
print("Fasttext dif: " + str(cosine_vectors(vectors_ft[0],vectors_ft[1])))
print("Bert dif: " + str(cosine_vectors(vectors_b[0],vectors_b[1])))

In [ ]:
from decimal import Decimal
rep = '%.4E' % Decimal(88799777378)
print(rep)

In [ ]:
df = pd.DataFrame(vectors, columns=["colummn"])
df.to_csv('lit_vectors.csv', index=False)

In [ ]:
len(lit_nodes)